OBTAINING AND FILTERING DATA

In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

2022 was the material rule change that affected games (As of this book). Othter seasons are natural breaking points are 2002, 2011, 2015, 2020, and 2021.

In [4]:
# use the range() function to load the NFL seasons (or years) we want to look into.
seasons = range(2016, 2022 + 1)
pbp_py = nfl.import_pbp_data(seasons)

2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


To get the subset of data you need for this analysis, filter down to just the passing plays.

Play_type being equal to pass will eliminate both running plays and plays that are negated because of penalty.
Sometimes you want to include plays that have penality (i.e. if you are using a grade-based system like the one at PFF).
Grade-based systems attempt to measure how well the player performed on a play independent of the final statistics of the play, so keeping data where play_type == no_ply have value.

In [ ]:
# to omit plays is air_yards is NA (in R) or NULL (in Python)

In [10]:
pbp_py_p = pbp_py \
            .query("play_type == 'pass' & air_yards.notnull()") \
            .reset_index()

DATA CLEANING AND WRANGLING

In [13]:
# define "long" pass as a pass that has air yards greater than or equal to 20 yards
# define "short" pass as one with air yards less than 20 yards.
# The NFL has a categorical variable for pass length (pass_length) in data, but the classifications are not completely clear to the observer.

# passing yards for incomplete passes are recorded as NA in R, or NULL in Python, but should be set to 0 for this analysis

# where() function from numpy will help us with this change.

In [11]:
pbp_py_p["pass_length_air_yards"] = np.where(
    pbp_py_p["air_yards"] >= 20, "long", "short"
)

In [12]:
pbp_py_p["passing_yards"] = \
                                np.where(
                                    pbp_py_p["passing_yards"].isnull(), 0, pbp_py_p["passing_yards"]
                                    )

SUMMARIZING DATA

In [18]:
pbp_py_p["passing_yards"]\
    .describe()

count    131606.000000
mean          7.192111
std           9.667647
min         -20.000000
25%           0.000000
50%           5.000000
75%          11.000000
max          98.000000
Name: passing_yards, dtype: float64

observe summary of data under different values of pass_length_air_yards
short passes, filter out the long passes and then summarize.

In [15]:
pbp_py_p\
    .query('pass_length_air_yards == "short"')["passing_yards"]\
    .describe()

count    116087.000000
mean          6.526812
std           7.695791
min         -20.000000
25%           0.000000
50%           5.000000
75%          10.000000
max          95.000000
Name: passing_yards, dtype: float64

find long passes in Python

The interquartile range (difference between 1st and 3rd quartile is much larger for longer pases than for short passes), even though the maximum passing yards are about the same.

The minimum values are going to be higher for long passes, since it's almost impossible to gain negative yards on a pass that travels 20 or more yards in the air.

In [20]:
pbp_py_p\
    .query('pass_length_air_yards == "long"')["passing_yards"]\
    .describe()

count    15519.000000
mean        12.168761
std         17.923370
min          0.000000
25%          0.000000
50%          0.000000
75%         26.000000
max         98.000000
Name: passing_yards, dtype: float64

expected points added (EPA), which are a more continous measure of play success that uses situtiational factors to assign a point value to each play.

In [23]:
# can use epa for long and short passes
pbp_py_p\
    .query('pass_length_air_yards == "short"')["epa"]\
    .describe()

count    116086.000000
mean          0.119606
std           1.426229
min         -13.031219
25%          -0.606135
50%          -0.002100
75%           0.959107
max           8.241420
Name: epa, dtype: float64

NameError: name 'my_plot' is not defined